In [65]:
import numpy as np
import pandas as pd
import itertools

import matplotlib.pyplot as plt

In [29]:
def f(X):
    np.random.seed(0)
    beta = np.random.rand(X.shape[-1])
    return np.dot(X, beta) + 10


def powerset(iterable):
    s = list(iterable)
    return itertools.chain.from_iterable(
        itertools.combinations(s, r) for r in range(len(s) + 1)
    )

In [2]:
M = 4
np.random.seed(1)
x = np.random.randn(M)

In [8]:
beta = np.random.rand(x.shape[-1])
f_x = np.dot(x, beta) + 10

In [31]:
reference = np.zeros(M)

X = np.zeros((2**M, M + 1))
X[:, -1] = 1
weights = np.zeros(2**M)

V = np.zeros((2**M, M))
for i in range(2**M):
    V[i, :] = reference

ws = {}
for i, s in enumerate(powerset(range(M))):
    s = list(s)
    V[i, s] = x[s]
    X[i, s] = 1
    # ws[len(s)] = ws.get(len(s), 0) + shapley_kernel(M, len(s))
    # weights[i] = shapley_kernel(M, len(s))

In [32]:
V

array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 1.62434536,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.61175641,  0.        ,  0.        ],
       [ 0.        ,  0.        , -0.52817175,  0.        ],
       [ 0.        ,  0.        ,  0.        , -1.07296862],
       [ 1.62434536, -0.61175641,  0.        ,  0.        ],
       [ 1.62434536,  0.        , -0.52817175,  0.        ],
       [ 1.62434536,  0.        ,  0.        , -1.07296862],
       [ 0.        , -0.61175641, -0.52817175,  0.        ],
       [ 0.        , -0.61175641,  0.        , -1.07296862],
       [ 0.        ,  0.        , -0.52817175, -1.07296862],
       [ 1.62434536, -0.61175641, -0.52817175,  0.        ],
       [ 1.62434536, -0.61175641,  0.        , -1.07296862],
       [ 1.62434536,  0.        , -0.52817175, -1.07296862],
       [ 0.        , -0.61175641, -0.52817175, -1.07296862],
       [ 1.62434536, -0.61175641, -0.52817175, -1.07296862]])

# Trial on wing weight

In [36]:
import sys
dirname = '/Users/muhammaddaffarobani/Documents/personal_research/smt'
if dirname not in sys.path:
    sys.path.append(dirname)

import pandas as pd
import numpy as np
from smt.surrogate_models import KRG
from smt.problems import WingWeight
from smt.sampling_methods import LHS
from sklearn.metrics import mean_squared_error


from smt.applications.explainability_tools import (
    PartialDependenceDisplay, 
    PDFeatureImportanceDisplay, 
    PDFeatureInteractionDisplay,
    ShapFeatureImportanceDisplay,
    ShapDisplay,
    ShapDisplay2,
    individual_shap_values,
)


In [34]:
ndoe = 100 
fun = WingWeight()
sampling = LHS(xlimits=fun.xlimits, criterion='ese', random_state=1)
X = sampling(ndoe)
y = fun(X)

n_train = int(0.8 * ndoe)

X_tr, y_tr = X[:n_train, :], y[:n_train]
X_te, y_te = X[n_train:, :], y[n_train:]

## config
feature_names = [
    r'$S_{w}$', r'$W_{fw}$', r'$A$', r'$\Delta$', 
    r'$q$', r'$\lambda$', r'$t_{c}$', r'$N_{z}$', 
    r'$W_{dg}$', r'$W_{p}$',
]

class GroundTruthModel:
    def predict_values(self, X):
        return fun(X)

In [35]:
print("Ground truth model")
gtm = GroundTruthModel()
y_pred = gtm.predict_values(X_te)
rmse = mean_squared_error(y_te, y_pred, squared=False)
rrmse = rmse / y_te.mean()
print(f"RMSE: {rmse:.4f}")
print(f"rRMSE: {rrmse:.4f}")

Ground truth model
RMSE: 0.0000
rRMSE: 0.0000


In [38]:
shap_values_1 = individual_shap_values(X_te[:5], gtm, X_tr, [False]*X_tr.shape[1])

In [39]:
shap_values_1

array([[ 2.37668078e+01,  1.68743274e-01,  1.59515435e+01,
         8.02488253e-01, -3.13393033e-01,  2.37548710e+00,
        -8.04822153e+00, -1.84499795e+01,  1.56197187e+01,
        -3.65090610e+00],
       [-2.63070026e+00,  1.07122252e-01, -1.03923668e+01,
         5.64212813e-02,  6.21939170e-01, -1.36788962e+00,
         4.06419521e+01,  3.78713190e+01, -1.63356262e+01,
         3.54943294e+00],
       [-6.79674047e+00, -1.06472376e-03,  3.05940607e+01,
         3.77508202e-01, -3.06589460e-01, -9.62389046e-01,
        -2.40620032e+01,  4.50441700e+01,  1.08620205e+01,
         2.82538276e+00],
       [ 2.83322403e+01, -8.77251607e-02,  2.69903986e+01,
         8.18161170e-02, -1.04175469e+00,  1.16413727e+00,
        -2.73688780e+00, -7.05879459e+00, -1.00162585e+01,
        -1.69430453e+00],
       [ 2.89330857e+01, -4.75904746e-02,  1.21505437e+01,
         1.55365319e+00,  4.42606923e-01,  3.11162822e+00,
         4.81343356e+00, -3.54456560e+00, -9.37929990e+00,
        -3.

In [107]:
from itertools import product
from scipy.special import comb

def create_mask_array(m):
    mask = np.array(list(product(range(2), repeat=m)))
    # remove mask where all elements are 0 / 1
    mask = mask[
        (~np.all(mask == 0, axis=1)) &
        (~np.all(mask == 1, axis=1))
    ]
    return mask

def get_reference_feature_values(x, is_categorical):
    # get reference values for each feature
    # if the feature is categorical/ordinal -> mode
    # else -> mean
    num_features = x.shape[1]
    reference_values = np.zeros(num_features)
    for feature_idx in range(num_features):
        if is_categorical[feature_idx] == 1:
            # mode = stats.mode(x[:, feature_idx], keepdims=False)[0]
            # reference_values[feature_idx] = mode
            reference_values[feature_idx] = np.random.choice(x[:, feature_idx])
        else:
            mean = np.mean(x[:, feature_idx])
            reference_values[feature_idx] = mean
    return reference_values


def get_reference_feature_values_2(x):
    # get reference values for each feature
    # if the feature is categorical/ordinal -> mode
    # else -> mean
    num_features = x.shape[1]
    reference_values = np.zeros(num_features)
    for feature_idx in range(num_features):
        reference_values[feature_idx] = np.random.choice(x[:, feature_idx])
    return reference_values


def calculate_weight(mask_row):
    m = len(mask_row)
    z = np.sum(mask_row)
    numerator = m - 1
    denominator = comb(m, z) * z * (m - z)
    weight = numerator / denominator
    return weight


def compute_shap_values(
    mask,
    s_full,
    weights,
    reference_values,
    model,
):
    y = model.predict_values(s_full)
    # b0 = model.predict_values(reference_values.reshape(1, -1))
    b0 = model.predict_values(reference_values)
    y = y - b0

    w = np.diag(weights)

    b = np.dot(
        np.linalg.inv(np.dot(np.dot(mask.transpose(), w), mask)),
        np.dot(np.dot(mask.transpose(), w), y)
    )
    b = b.reshape(-1, )
    return b


def compute_shap_values_2(
    mask,
    s_full,
    weights,
    model,
):
    y = model.predict_values(s_full)
    # b0 = model.predict_values(reference_values.reshape(1, -1))
    # b0 = model.predict_values(reference_values)
    # y = y - b0
    mask_2 = np.ones((mask.shape[0], mask.shape[1]+1))
    mask_2[:, :-1] = mask

    w = np.diag(weights)
    w = np.sqrt(w)

    b = np.dot(
        np.linalg.inv(np.dot(np.dot(mask_2.transpose(), w), mask_2)),
        np.dot(np.dot(mask_2.transpose(), w), y)
    )
    b = b.reshape(-1, )
    b = b[:-1]

    return b



def individual_shap_values_2(
    instances,
    model,
    x,
    is_categorical,
):
    # reference_values = get_reference_feature_values(x, is_categorical)
    shap_values = list()

    for instance in instances:
        instance = instance.reshape(1, -1)
        mask = create_mask_array(instance.shape[1])
        mask = np.repeat(mask, 10, axis=0)
        # s_with_zero = mask * instance
        reference_values = np.ones(mask.shape)
        for i in range(len(reference_values)):
            reference_values[i, :] = get_reference_feature_values_2(x)
        # s_full = (s_with_zero == 0) * reference_values + s_with_zero
        s_ref = (mask == 0) * reference_values
        s_real = (mask == 1) * instance
        s_full = s_ref + s_real

        weights = np.apply_along_axis(calculate_weight, 1, mask)
        shap_value = compute_shap_values_2(
            mask,
            s_full,
            weights,
            reference_values,
            model,
        )
        shap_values.append(shap_value)
    shap_values = np.array(shap_values)
    return shap_values


def individual_shap_values_3(instances, model, x):
    shap_values = list()

    for instance in instances:
        instance = instance.reshape(1, -1)
        mask = create_mask_array(instance.shape[1])
        mask = np.repeat(mask, 10, axis=0)
        weights = np.apply_along_axis(calculate_weight, 1, mask)

        s_full = np.where(mask == 1, instance, x[np.random.randint(x.shape[0])])

        shap_value = compute_shap_values_2(
            mask,
            s_full,
            weights,
            model,
        )
        shap_values.append(shap_value)
    shap_values = np.array(shap_values)
    return shap_values

    



In [105]:
shap_values_1 = individual_shap_values(X_tr, gtm, X_tr, [False]*X_tr.shape[1])

In [108]:
shap_values_2 = individual_shap_values_3(X_tr[:3], gtm, X_tr)

In [110]:
shap_values_1[:3]

array([[ 6.18330252e+00,  1.38281762e-01, -3.54616751e+01,
         4.59151659e-01,  6.57703342e-01, -4.06904366e-01,
        -2.08303277e+01,  2.76609129e+01,  1.58723781e+01,
        -2.72529511e+00],
       [-1.13174865e+01, -1.83463663e-01,  3.03998768e+01,
         9.42807913e-02, -3.71836877e-03, -4.76546767e+00,
         1.97117208e+01,  4.10660228e+01,  2.11564763e+01,
        -8.39920375e-01],
       [-1.04213763e+01, -1.10822681e-01, -9.22013000e+00,
         3.09935761e+00,  6.05037128e-01, -1.85815974e+00,
         2.76035664e+01,  1.06329658e+01,  1.14103418e+01,
        -4.75057108e-01]])

In [109]:
shap_values_2

array([[-3.59579444e+00,  6.90192377e-02, -1.77975203e+01,
        -2.69417740e+00,  9.46879091e-01, -3.52677227e+00,
        -3.44649888e+01, -2.62471540e+00,  1.76848568e+01,
        -6.85952011e-01],
       [-3.33997087e+01,  2.15347866e-02,  6.57951404e+01,
        -1.04189772e+00, -7.76108169e-02, -6.28167544e+00,
         3.99072326e+01,  3.46674105e+01,  4.06053125e+01,
        -1.44260959e+00],
       [-5.11558998e+00, -2.69785454e-01, -3.96269777e+01,
         3.57596749e+00,  1.81612084e+00, -4.34103544e+00,
         1.37709806e+01, -3.62981602e+01,  1.82430328e+00,
         1.37179296e+00]])